# CIFAR-100 Classification with ResNet34

This notebook implements a modified ResNet34 architecture for CIFAR-100 classification, targeting 75% accuracy. The implementation includes several optimizations:

- Adapted ResNet34 architecture for 32x32 images
- Strong data augmentation strategy
- One Cycle Learning Rate policy with warmup
- Label smoothing and gradient clipping
- Memory optimizations for efficient training

## Training Configuration (Iteration 2)

Key improvements in this iteration:
1. Removed MaxPooling layers, using stride>1 convolutions for downsampling
2. Enhanced data augmentation pipeline
3. Added label smoothing (0.1) for better generalization
4. Implemented OneCycleLR with 10% warmup phase
5. Memory optimizations including gradient clipping and periodic cache clearing

In [ ]:
import torch
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from model_2 import ResNet34 as Net
from train_2 import train, train_losses, train_acc
from test import test, test_losses, test_acc
from torchsummary import summary
import torchvision
import torch.nn as nn

In [5]:
# Data transforms (use helpers from train.py)
# from train_2 import train_transforms, test_transforms

# Set augment=True to enable small augmentations during training
train_transforms = train_transforms(augment=True)
test_transforms = test_transforms()

In [6]:
print("\nDownloading and loading CIFAR-100 dataset...")

# Dataset and DataLoader
train_dataset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=train_transforms)
test_dataset  = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=test_transforms)

# Reduce batch size to save memory
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=2, pin_memory=True)

100%|██████████| 169M/169M [00:03<00:00, 43.5MB/s] 


In [ ]:
LEARNING_RATE = 0.1
MOMENTUM = 0.9
WEIGHT_DECAY = 0.0005  # Reduced for better generalization
EPOCHS = 300

# Model, device, optimizer
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)

criterion = nn.CrossEntropyLoss()

# Optimizer with improved settings
optimizer = optim.SGD(
    model.parameters(), 
    lr=LEARNING_RATE, 
    momentum=MOMENTUM, 
    weight_decay=WEIGHT_DECAY, 
    nesterov=True
)

# Enhanced Learning Rate Scheduler with warmup
from torch.optim.lr_scheduler import OneCycleLR


scheduler = OneCycleLR(
    optimizer,
    max_lr=LEARNING_RATE,
    epochs=EPOCHS,
    steps_per_epoch=len(train_loader),
    pct_start=0.1,  # 10% warmup
    div_factor=10,  # Initial LR = max_lr/div_factor
    final_div_factor=100  # Final LR = max_lr/final_div_factor
)

summary(model, input_size=(3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
       BatchNorm2d-2           [-1, 64, 32, 32]             128
              ReLU-3           [-1, 64, 32, 32]               0
            Conv2d-4           [-1, 64, 32, 32]          36,864
       BatchNorm2d-5           [-1, 64, 32, 32]             128
              ReLU-6           [-1, 64, 32, 32]               0
            Conv2d-7           [-1, 64, 32, 32]          36,864
       BatchNorm2d-8           [-1, 64, 32, 32]             128
              ReLU-9           [-1, 64, 32, 32]               0
           Conv2d-10           [-1, 64, 32, 32]          36,864
      BatchNorm2d-11           [-1, 64, 32, 32]             128
             ReLU-12           [-1, 64, 32, 32]               0
           Conv2d-13           [-1, 64, 32, 32]          36,864
      BatchNorm2d-14           [-1, 64,

In [ ]:

import os
import torch

checkpoint_path = "/kaggle/working/checkpoint.pth"

# --- Load checkpoint if it exists ---
start_epoch = 0
best_val_acc = 0.0

if os.path.exists(checkpoint_path):
    print(f"🔁 Found existing checkpoint at {checkpoint_path}. Resuming training...")
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    if 'scheduler_state_dict' in checkpoint:
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    start_epoch = checkpoint['epoch'] + 1
    best_val_acc = checkpoint.get('best_val_acc', 0.0)
    print(f"✅ Resumed from epoch {start_epoch}, best_val_acc={best_val_acc:.4f}")
else:
    print("🚀 No checkpoint found. Starting training from scratch.")

# --- Training Loop ---
num_epochs = 300  # Example; replace with yours

for epoch in range(start_epoch, EPOCHS):
    print(f"\nEPOCH: {epoch+1}/{EPOCHS}")
    
    # Clear memory before each epoch
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()
    
    # Training
    train(model, device, train_loader, optimizer, epoch, scheduler)
    
    # Testing
    test(model, device, test_loader)


    # --- Save checkpoint ---
    state = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'best_val_acc': best_val_acc,
    }
    if 'scheduler' in locals():
        state['scheduler_state_dict'] = scheduler.state_dict()

    torch.save(state, checkpoint_path)
    print(f"💾 Checkpoint saved at epoch {epoch+1}")

    # --- Optionally keep best model ---
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "/kaggle/working/best_model.pth")
        print(f"🏆 New best model saved (val_acc={val_acc:.2f}%)")


🚀 No checkpoint found. Starting training from scratch.

EPOCH: 1/300


Loss=4.037995338439941 Batch_id=390 Accuracy=5.79: 100%|██████████| 391/391 [00:43<00:00,  8.94it/s] 



Test set: Average loss: 3.6660, Accuracy: 1365/10000 (13.65%)

💾 Checkpoint saved at epoch 1

EPOCH: 2/300


Loss=3.838561534881592 Batch_id=390 Accuracy=13.99: 100%|██████████| 391/391 [00:43<00:00,  9.06it/s] 



Test set: Average loss: 3.2251, Accuracy: 2087/10000 (20.87%)

💾 Checkpoint saved at epoch 2

EPOCH: 3/300


...

Loss=1.1859322786331177 Batch_id=390 Accuracy=88.49: 100%|██████████| 391/391 [00:43<00:00,  9.07it/s]



Test set: Average loss: 1.0314, Accuracy: 7607/10000 (76.07%)

💾 Checkpoint saved at epoch 252

EPOCH: 253/300


Loss=1.2435898780822754 Batch_id=390 Accuracy=88.55: 100%|██████████| 391/391 [00:43<00:00,  9.05it/s]



Test set: Average loss: 1.0519, Accuracy: 7508/10000 (75.08%)

💾 Checkpoint saved at epoch 253

EPOCH: 254/300


Loss=1.2010140419006348 Batch_id=365 Accuracy=88.83:  94%|█████████▎| 366/391 [00:40<00:02,  9.42it/s]

In [ ]:
# Plotting results
import matplotlib.pyplot as plt
fig, axs = plt.subplots(2,2,figsize=(15,10))
axs[0, 0].plot([t.item() for t in train_losses])
axs[0, 0].set_title("Training Loss")
axs[1, 0].plot(train_acc)
axs[1, 0].set_title("Training Accuracy")
axs[0, 1].plot(test_losses)
axs[0, 1].set_title("Test Loss")
axs[1, 1].plot(test_acc)
axs[1, 1].set_title("Test Accuracy")
plt.show()